In [1]:
__author__ = "Brian Arbuckle, Benu Atri and Paras Jamil"
__version__ = "CS224u, Final Project 2021 v1"
__updated__ = "Mar 23, 2021, 11pmCST"

In [2]:
import os
import logging
from lxml import etree
import re
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime as dt

from parse_token_utils import create_directory, parse_fileName_vals
from filing_parsers import *

today = str(dt.today()).split(' ')[0]
today

'2021-03-25'

<div class="alert alert-info">
    <b>At some point</b><br> We will need to define what the labels are '1's and '0's as the discrepancy between performance_earnings and performance_filing this is true for about 10% - this is related to our hypothesis, the text that we need to show that it changes sentiment

In [51]:
## copied & adapted from data-engineering/parsers/filing_parsers.py (git)
forms = ['10-K']
def quick_parse(filepath):
    parser = etree.XMLParser(recover=True, huge_tree=True)
    tree = etree.parse(filepath, parser)
    notags = etree.tostring(tree, encoding='utf8', method='text')
    return notags
def parse_10q(filepath, cik, year):
    """
    Parses the 10-Q passed. Returns a dictionary:
        body: The body of the 10-Q
    :param filepath:
    :return:
    """
    from bs4 import BeautifulSoup
    results = {}
    with open(filepath) as fp:
        soup = BeautifulSoup(fp, "lxml")

    #results['TenKText'] = __extract_document_text(soup)
    results['headers'] = __parse_sec_header(soup, cik, year)
    results['documents'] = __parse_documents(soup)
    return results

def get_date(filingPath):
    """From FinBert"""
    #print(filingPath)
    accession_code = filingPath.split('/')[-2]
    ticker = filingPath.split('/')[-4]
    filing_type = filingPath.split('/')[-3]
    #print(accession_code, ticker, filing_type)
    code = accession_code+'.txt'
    with open(filingPath, 'r') as f:
        readlin = f.read()
    for i in readlin.split('\n'):
#         print(i)
        if code in i:
            date_extr = i.split(' : ')[1]
            return date_extr


def __parse_documents(soup):
    import time
    # Get all the douments
    result_documents = []
    documents = soup.find_all('document')

    for doc in documents:

        type_node = doc.find('type')
        if type_node:
            type_text = type_node.contents[0].strip()
            desc_node = type_node.find('description')
        else:
            type_text = 'NA'

        seq_node = doc.find('sequence')
        if seq_node:
            seq_text = seq_node.contents[0].strip()
        else:
            seq_text = str(int(time.time()))[-6:]

        if desc_node:
            desc_text = desc_node.contents[0]
        else:
            desc_text = ''

        if type_text in forms:
            is_10_q = True
        else:
            is_10_q = False

        if type_text not in ["XML", "GRAPHIC", "EXCEL", "ZIP"]:
            logging.debug("Parsing doucment type: {}".format(type_text))
            result_documents.append(
                {'is_10_q': is_10_q,
                 'type': type_text,
                 'sequence': seq_text,
                 'description': desc_text,
                 'document': __extract_document_text(soup)}
            )

    return result_documents

def __parse_sec_header(soup, cik, year):
    sec_header = soup.find("sec-header")

    if not sec_header:
        sec_header = soup

    result = {
        'cik': cik,
        'year': year,
    }
    result['accession_number'] = __get_line_item(sec_header, 'ACCESSION NUMBER:')
    result['conformed_period_of_report'] = __get_line_item(sec_header, 'CONFORMED PERIOD OF REPORT:')
    result['filed_as_of_date'] = __get_line_item(sec_header, 'FILED AS OF DATE:')
    result['company_confirmed_name'] = __get_line_item(sec_header, 'COMPANY CONFORMED NAME:')
    result['central_index_key'] = __get_line_item(sec_header, 'CENTRAL INDEX KEY:')
    result['standard_industrial_classification'] = __get_line_item(sec_header, 'STANDARD INDUSTRIAL CLASSIFICATION:')
    result['state_of_incorporation'] = __get_line_item(sec_header, 'STATE OF INCORPORATION:')
    result['fiscal_year_end'] = __get_line_item(sec_header, 'FISCAL QUARTER END:')

    return result

def __get_line_item(sec_header, attr_name):
    find_results = re.findall(attr_name + '(.*?)\n', str(sec_header))

    if find_results:
        return find_results[0].strip()
    else:
        return None
def __extract_document_text(soup):
    if len(soup) == 1:
        TenKtext = soup.find('text').extract().text
        TenKtext = parser_custom(TenKtext)
    #tables = document.find_all('table')
    #for table in tables:
    #    table.decompose()

    return TenKtext

In [4]:
FULL_DATASET = True
PICKLES = '../pickle_data/'
DATA = 'data'

# GLOBAL START AND END:
START = '2016-01-01'
END = '2021-03-01'

#doc_folder_name = 'item7_parsed_documents'
dataset_missing = 'needs_work/'
if FULL_DATASET == True:
    stocks_PATH = os.path.join(PICKLES, "performance_labeled.pickle")
forms = ['10-Q']

In [5]:
#for i in os.listdir(dataset_missing):
#    print(i)

In [6]:
missing_list = ["AMAT", "CMCSA", "EA", "ILMN"]
stocks_PATH

'../pickle_data/performance_labeled.pickle'

In [8]:
stocks_df = pd.read_pickle(stocks_PATH) 
list_of_files = ["../data/sec-edgar-filings/AMAT/10-K/0000006951-18-000041/full-submission.txt", "../data/sec-edgar-filings/AMAT/10-K/0000006951-17-000038/full-submission.txt", "../data/sec-edgar-filings/AMAT/10-K/0000006951-16-000068/full-submission.txt", "../data/sec-edgar-filings/CMCSA/10-K/0001166691-20-000008/full-submission.txt", "../data/sec-edgar-filings/CMCSA/10-K/0001166691-19-000005/full-submission.txt", "../data/sec-edgar-filings/CMCSA/10-K/0001166691-18-000004/full-submission.txt", "../data/sec-edgar-filings/CMCSA/10-K/0001193125-17-030512/full-submission.txt", "../data/sec-edgar-filings/CMCSA/10-K/0001193125-16-452423/full-submission.txt", "../data/sec-edgar-filings/EA/10-K/0000712515-20-000019/full-submission.txt", "../data/sec-edgar-filings/EA/10-K/0000712515-19-000019/full-submission.txt", "../data/sec-edgar-filings/EA/10-K/0000712515-18-000024/full-submission.txt", "../data/sec-edgar-filings/EA/10-K/0000712515-17-000035/full-submission.txt", "../data/sec-edgar-filings/EA/10-K/0000712515-16-000111/full-submission.txt", "../data/sec-edgar-filings/ILMN/10-K/0001110803-20-000018/full-submission.txt"]
stocks_df = stocks_df[stocks_df['ticker'].isin(missing_list)]

In [9]:
stocks_df.head()
parsed_sec_filings_path = '../data/2021-03-20_parsed_sec_filings_path/'

In [11]:
stocks_df.ticker.unique()
with open('failed_parser.txt', 'r') as f:
    failed_files = f.readlines()
failed_files = [x.strip() for x in failed_files]
failed_files = list_of_files

In [14]:
failed_files
s = '/home/jovyan/work/benu_work/needs_work/ILMN/10-K/0001110803-20-000018/full-submission.txt'
print('../data/sec-edgar-filings/'+ s.split('needs_work/')[1])

failed_files

../data/sec-edgar-filings/ILMN/10-K/0001110803-20-000018/full-submission.txt


['../data/sec-edgar-filings/AMAT/10-K/0000006951-18-000041/full-submission.txt',
 '../data/sec-edgar-filings/AMAT/10-K/0000006951-17-000038/full-submission.txt',
 '../data/sec-edgar-filings/AMAT/10-K/0000006951-16-000068/full-submission.txt',
 '../data/sec-edgar-filings/CMCSA/10-K/0001166691-20-000008/full-submission.txt',
 '../data/sec-edgar-filings/CMCSA/10-K/0001166691-19-000005/full-submission.txt',
 '../data/sec-edgar-filings/CMCSA/10-K/0001166691-18-000004/full-submission.txt',
 '../data/sec-edgar-filings/CMCSA/10-K/0001193125-17-030512/full-submission.txt',
 '../data/sec-edgar-filings/CMCSA/10-K/0001193125-16-452423/full-submission.txt',
 '../data/sec-edgar-filings/EA/10-K/0000712515-20-000019/full-submission.txt',
 '../data/sec-edgar-filings/EA/10-K/0000712515-19-000019/full-submission.txt',
 '../data/sec-edgar-filings/EA/10-K/0000712515-18-000024/full-submission.txt',
 '../data/sec-edgar-filings/EA/10-K/0000712515-17-000035/full-submission.txt',
 '../data/sec-edgar-filings/EA/

In [13]:
stocks_df

,ticker,cik,FiscalQuarter End,DateReported,Time,open,close,performance_earnings,form,filing,date_filed,open_2,performance_filing
134,AMAT,6951,Jul2020,2020-08-13,After Close,2020-08-14,2020-11-12,0,10-Q,Quarterly report,2020-08-20,2020-08-21,1
135,AMAT,6951,Apr2020,2020-05-14,After Close,2020-05-15,2020-08-13,1,10-Q,Quarterly report,2020-05-21,2020-05-22,1
136,AMAT,6951,Jan2020,2020-02-12,After Close,2020-02-13,2020-05-14,0,10-Q,Quarterly report,2020-02-20,2020-02-21,0
137,AMAT,6951,Oct2019,2019-11-14,After Close,2019-11-15,2020-02-12,0,10-K,Annual report,2019-12-13,2019-12-16,0
138,AMAT,6951,Jul2019,2019-08-15,After Close,2019-08-16,2019-11-14,1,10-Q,Quarterly report,2019-08-22,2019-08-23,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,ILMN,1110803,Dec2016,2017-01-31,After Close,2017-02-01,2017-04-25,1,10-K,Annual report,2017-02-14,2017-02-15,1
696,ILMN,1110803,Sep2016,2016-11-01,After Close,2016-11-02,2017-01-31,1,10-Q,Quarterly report,2016-11-08,2016-11-09,1
697,ILMN,1110803,Jun2016,2016-07-26,After Close,2016-07-27,2016-11-01,0,10-Q,Quarterly report,2016-08-02,2016-08-03,0
698,ILMN,1110803,Mar2016,2016-05-03,After Close,2016-05-04,2016-07-26,1,10-Q,Quarterly report,2016-05-09,2016-05-10,0


In [22]:
from bs4 import BeautifulSoup
#sec_filings_path = '/home/jovyan/work/benu_work/needs_work/'
sec_filings_path = '/home/jovyan/work/data/sec-edgar-filings/'
def run_parse(stocks_df):

    num_stocks = stocks_df.shape[0]
    c = 0

    for idx, row in stocks_df.iterrows():
        #        if row['form'] in ['10-Q','10-K']:

        if row['form'] in ['10-Q','10-K']:
            ticker_form_path = os.path.join(sec_filings_path, row['ticker'], row['form'][:4])
            
            ciks = os.listdir(ticker_form_path)
            for cik in ciks:
                if cik == '.DS_Store':
                    continue
               
                #text_to_parse = os.path.join(ticker_form_path, cik, 'full-submission_original.txt')
                text_to_parse = os.path.join(ticker_form_path, cik, 'full-submission-Copy1.txt')                    
                if not os.path.exists(text_to_parse):
                    continue
                
                if row['date_filed'] == dt.strptime(get_date(text_to_parse), '%Y%m%d'):
                    c += 1
                    print("{:0>2d} of {} {}".format(c, num_stocks, (30*'-')))
                    ticker, cik_10dig, filing_type, filing_qrtr, doc_reported_on, doc_year = parse_fileName_vals(cik, row, text_to_parse)
                    print("ticker, cik_10dig, filing_type, filing_qrtr, doc_reported_on, doc_year DONE")
                    out_file_name = ticker+'_'+cik_10dig+'_'+filing_type+'_'+filing_qrtr+ ".txt"
                    out_file_path = os.path.join(parsed_sec_filings_path, out_file_name)


                    to_actually_parse = os.path.join(ticker_form_path, cik, 'full-submission.txt')
                    print(to_actually_parse)
                    print('hi')
                    #s = '../data/sec-edgar-filings/'+ to_actually_parse.split('needs_work/')[1]
                    #print(s)
                    #print('found here', failed_files.index(s))

                    #if s not in failed_files:
                    #    continue
                    print('FOUND IN FAILED FILES', to_actually_parse)
                    with open(to_actually_parse, 'r') as fp:
                        soup = BeautifulSoup(fp, "html.parser")
                        print(cik, len(soup))

                        TenKItem7 = soup.text
                        TenKItem7 = TenKItem7.strip() # Remove starting/ending white spaces
                        TenKItem7 = TenKItem7.replace('\n', ' ') # Replace \n (new line) with space
                        TenKItem7 = TenKItem7.replace('\r', '') # Replace \r (carriage returns-if you're on windows) with space
                        TenKItem7 = TenKItem7.replace(' ', ' ') # Replace " " (a special character for space in HTML) with space
                        TenKItem7 = TenKItem7.replace(' ', ' ') # Replace " " (a special character for space in HTML) with space
                        TenKItem7 = TenKItem7.replace(u'/xa0', u' ') # 
                        while '  ' in TenKItem7:
                            TenKItem7 = TenKItem7.replace('  ', ' ') # Remove extra spaces



                        pattern = '0.0);text-align:right;vertical-align:bottom;padding-left:1pt;padding-right:1pt;">'

                        TenKItem7 = TenKItem7.replace(pattern, '')
                        pattern2 = r'style="line-height:120%;font-size:10pt;padding-left:0px;">'
                        #'0.0);text-align:right;vertical-align:bottom;padding-left:1pt;padding-right:1pt;">'
                        pattern3 = r'•'
                        pattern4 = 'style="background-color:rgb(255,255,255, 0.0);color:#000000;font-family:\'Arial\',sans-serif;font-size:10pt;font-weight:700;line-height:120%;">'
                        TenKItem7 = TenKItem7.replace(pattern2, '')
                        TenKItem7 = TenKItem7.replace(pattern3, '')
                        TenKItem7 = TenKItem7.replace(pattern4, '')
                        pattern5 = r'12.00pt"> '
                        TenKItem7 = TenKItem7.replace(pattern4, '')
                        pattern6 = r'style="margin-top: 0; margin-bottom: 0; text-align: left">'
                        pattern8 = r'0.0);color:#000000;font-family:\'Arial\',sans-serif;font-size:11pt;font-weight:700;line-height:120%;">'
                        pattern7 = r'style="line-height:120%;font-size:11pt;">'
                        TenKItem7 = TenKItem7.replace(pattern5, '')
                        TenKItem7 = TenKItem7.replace(pattern6, '')
                        TenKItem7 = TenKItem7.replace(pattern7, '')
                        TenKItem7 = TenKItem7.replace(pattern8, '')
                        with open(out_file_path, 'w') as f:

                            f.write(TenKItem7)
                            print()
                        print(f'Parsing finished, output saved in:\n{out_file_path}')
                        print('__'*30)
                        print()
                        if c%10==0:
                            input()

run_parse(stocks_df)

01 of 76 ------------------------------
ticker, cik_10dig, filing_type, filing_qrtr, doc_reported_on, doc_year DONE
/home/jovyan/work/data/sec-edgar-filings/AMAT/10-K/0000006951-18-000041/full-submission.txt
hi
FOUND IN FAILED FILES /home/jovyan/work/data/sec-edgar-filings/AMAT/10-K/0000006951-18-000041/full-submission.txt
0000006951-18-000041 309

Parsing finished, output saved in:
../data/2021-03-20_parsed_sec_filings_path/AMAT_0000006951_10-K_Oct2018.txt
____________________________________________________________

02 of 76 ------------------------------
ticker, cik_10dig, filing_type, filing_qrtr, doc_reported_on, doc_year DONE
/home/jovyan/work/data/sec-edgar-filings/AMAT/10-K/0000006951-17-000038/full-submission.txt
hi
FOUND IN FAILED FILES /home/jovyan/work/data/sec-edgar-filings/AMAT/10-K/0000006951-17-000038/full-submission.txt
0000006951-17-000038 322

Parsing finished, output saved in:
../data/2021-03-20_parsed_sec_filings_path/AMAT_0000006951_10-K_Oct2017.txt
______________

In [12]:
#


soup.text

'0.0);text-align:right;vertical-align:bottom;padding-left:1pt;padding-right:1pt;">Alphabet Inc.ITEM 2.MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONSPlease read the following discussion and analysis of our financial condition and results of operations together with our consolidated financial statements and related notes included under Part I, Item\xa01 of this Quarterly Report on Form 10-Q.Executive Overview of ResultsBelow are our key financial results for the three months ended June 30, 2020 (consolidated unless otherwise noted):•Revenues were $38.3 billion, a decrease of 2% year over year, constant currency revenues were flat year over year.•Google segment revenues were $38.0 billion, a decrease of 2% year over year, and Other Bets revenues were $148 million, a decrease of 9% year over year.•Revenues from the United States, EMEA, APAC, and Other Americas were $18.0 billion, $11.4 billion, $6.9 billion, and $1.8 billion, respectively.•Cost of rev

In [18]:

def parser_custom(TenKtext):
    # Clean newly extracted text
    TenKItem7 = TenKItem7.strip() # Remove starting/ending white spaces
    TenKItem7 = TenKItem7.replace('\n', ' ') # Replace \n (new line) with space
    TenKItem7 = TenKItem7.replace('\r', '') # Replace \r (carriage returns-if you're on windows) with space
    TenKItem7 = TenKItem7.replace(' ', ' ') # Replace " " (a special character for space in HTML) with space
    TenKItem7 = TenKItem7.replace(' ', ' ') # Replace " " (a special character for space in HTML) with space

    while '  ' in TenKItem7:
        TenKItem7 = TenKItem7.replace('  ', ' ') # Remove extra spaces

    # Print first 500 characters of newly extracted text
    #print(TenKItem7[:500])
    TenKItem7 = TenKItem7.replace(u'/xa0', u' ') # 
    return TenKItem7

In [ ]:
'''discards = []
def parser_custom(TenKtext):
    
    matches = re.compile(r'(item\s(7[\.\s]|8[\.\s])|'
                         'discussion\sand\sanalysis\sof\s(consolidated\sfinancial|financial)\scondition|'
                         '(consolidated\sfinancial|financial)\sstatements\sand\ssupplementary\sdata)', re.IGNORECASE)

    matches_array = pd.DataFrame([(match.group(), match.start()) for match in matches.finditer(TenKtext)])
    print(len(matches_array), matches_array.shape, matches_array.columns)
    # Set columns in the dataframe
    matches_array.columns = ['SearchTerm', 'Start']
    print('hi', len(matches_array), matches_array.shape, matches_array.columns)
    # Get the number of rows in the dataframe
    Rows = matches_array['SearchTerm'].count()
    
    # Create a new column in 'matches_array' called 'Selection' and add adjacent 'SearchTerm' (i and i+1 rows) text concatenated
    count = 0 # Counter to help with row location and iteration
    while count < (Rows-1): # Can only iterate to the second last row
        matches_array.at[count,'Selection'] = (matches_array.iloc[count,0] + matches_array.iloc[count+1,0]).lower() # Convert to lower case
        count += 1

    # Set up 'Item 7/8 Search Pattern' regex patterns
    matches_item7 = re.compile(r'(item\s7\.discussion\s[a-z]*)')
    matches_item8 = re.compile(r'(item\s8\.(consolidated\sfinancial|financial)\s[a-z]*)')

    # Lists to store the locations of Item 7/8 Search Pattern matches
    Start_Loc = []
    End_Loc = []

    # Find and store the locations of Item 7/8 Search Pattern matches
    count = 0 # Set up counter

    while count < (Rows-1): # Can only iterate to the second last row

        # Match Item 7 Search Pattern
        if re.match(matches_item7, matches_array.at[count,'Selection']):
            # Column 1 = 'Start' column in 'matches_array'
            
            Start_Loc.append(matches_array.iloc[count,1]) # Store in list => Item 7 will be the starting location (column '1' = 'Start' column)

        # Match Item 8 Search Pattern
        if re.match(matches_item8, matches_array.at[count,'Selection']):
            End_Loc.append(matches_array.iloc[count,1])

        count += 1
    
    print(Start_Loc, End_Loc)
    input()
    if len(Start_Loc)==len(End_Loc)==0:
        return False
    if len(Start_Loc)==len(End_Loc)==1:
        print(Start_Loc, End_Loc)
    

        # Extract section of text and store in 'TenKItem7'
        TenKItem7 = TenKtext[Start_Loc[0]:End_Loc[0]]
        #print(len(TenKItem7))
    elif len(Start_Loc)!=len(End_Loc):
        TenKItem7 = TenKtext[Start_Loc[0]:End_Loc[-1]]
    else :
        print(Start_Loc, End_Loc)
        # Extract section of text and store in 'TenKItem7'
        TenKItem7 = TenKtext[Start_Loc[1]:End_Loc[1]]
        #print(len(TenKItem7))
        
    # Clean newly extracted text
    TenKItem7 = TenKItem7.strip() # Remove starting/ending white spaces
    TenKItem7 = TenKItem7.replace('\n', ' ') # Replace \n (new line) with space
    TenKItem7 = TenKItem7.replace('\r', '') # Replace \r (carriage returns-if you're on windows) with space
    TenKItem7 = TenKItem7.replace(' ', ' ') # Replace " " (a special character for space in HTML) with space
    TenKItem7 = TenKItem7.replace(' ', ' ') # Replace " " (a special character for space in HTML) with space

    while '  ' in TenKItem7:
        TenKItem7 = TenKItem7.replace('  ', ' ') # Remove extra spaces

    # Print first 500 characters of newly extracted text
    #print(TenKItem7[:500])
    TenKItem7 = TenKItem7.replace(u'/xa0', u' ') # 
    return TenKItem7'''